In [24]:
import os,sys,glob
import numpy as np
import random
from itertools import combinations
import pandas as pd
from scipy.stats import percentileofscore

### Average final results across data folds
goes from (n_runs, n_subjects, n_features) -> (n_subjects, n_features)
and we can average across features to get an average value (barplots) or across subjects to get a feature map

In [1]:
for dataset in ['budapest','raiders','whiplash']:
    rha_dir = '../../response_hyper/{}/'.format(dataset)
    for metric in ["dense","vertex_isc","clf"]:
        outfn = os.path.join('../final_results/{}/rha_{}.npy'.format(dataset,metric))
        g = glob.glob(os.path.join(rha_dir, "fold*","results","*{}*.npy".format(metric)))
        metric_results = np.stack([np.load(b) for b in g])
        avg_result = np.mean(metric_results, axis=0)
        np.save(outfn, avg_result)

    cha_dir = '../../conn_hyper/{}/'.format(dataset)
    for metric in ["dense","vertex_isc","clf"]:
        outfn = os.path.join('../final_results/{}/cha_{}.npy'.format(dataset,metric))
        g = glob.glob(os.path.join(cha_dir, "fold*","results","*{}*.npy".format(metric)))
        metric_results = np.stack([np.load(b) for b in g])
        avg_result = np.mean(metric_results, axis=0)
        np.save(outfn, avg_result)

    h2a_dir = '../../response_hyper/{}/'.format(dataset)
    for metric in ["dense","vertex_isc","clf"]:
        outfn = os.path.join('../final_results/{}/h2a_{}.npy'.format(dataset,metric))
        g = glob.glob(os.path.join(h2a_dir, "fold*","results","*{}*.npy".format(metric)))
        metric_results = np.stack([np.load(b) for b in g])
        avg_result = np.mean(metric_results, axis=0)
        np.save(outfn, avg_result)

    aa_dir = '../../response_hyper/{}/'.format(dataset)
    for metric in ["dense","vertex_isc","clf"]:
        outfn = os.path.join('../final_results/{}/aa_{}.npy'.format(dataset,metric))
        g = glob.glob(os.path.join(aa_dir, "fold*","results","*{}*.npy".format(metric)))
        metric_results = np.stack([np.load(b) for b in g])
        avg_result = np.mean(metric_results, axis=0)
        np.save(outfn, avg_result)

NameError: name 'os' is not defined

In [25]:
results_dict = {'dataset':[],'metric':[],'comparison':[],'pvalue':[]}

In [27]:
# observed_difference = model1_metricA - model2_metricA
def permutation_test(observed_difference, n_iterations):
    obtained_mean = observed_difference.mean()
    null_distribution_of_means = np.ndarray((n_iterations))
    # flip sign permutation test
    for i in range(n_iterations):
        weights = [random.choice([1,-1]) for d in range(len(observed_difference))]
        null_distribution_of_means[i]=(weights * observed_difference).mean()
    percentile = percentileofscore(null_distribution_of_means, obtained_mean)
    return percentile # this returns the percentile of obtained score versus the null distribution
# of scores. Then we compute the pvalue depending upon if it's a 1 tailed or 2 tailed value.

def run_permutations(data, n_iterations):
    pvalues = {}
    combos = combinations(data.keys(),2) 
    for combo in combos:
        combo_label = str(combo[0])+'_'+str(combo[1])
        obtained_difference = data[combo[0]] - data[combo[1]]
        percentile = permutation_test(obtained_difference, n_iterations)
        if (obtained_difference>0).sum() < 20:
            percentile=100.-percentile
        pvalues[combo_label] = (100.-percentile)/100.
    return pvalues

### Budapest significance testing

In [6]:
budapest_results = '../../final_results/budapest/'
isc={}
for A in ['aa','cha','rha','h2a']:
    isc[A] = np.mean(np.nan_to_num(np.load(os.path.join(budapest_results, '{A}_vertex_isc.npy'.format(A=A)))),axis=0)

clf={}
for A in ['aa','cha','rha','h2a']:
    clf[A] = np.mean(np.nan_to_num(np.load(os.path.join(budapest_results, '{A}_clf.npy'.format(A=A)))),axis=1)

cnx={}
for A in ['aa','cha','rha','h2a']:
    cnx[A] = np.mean(np.nan_to_num(np.load(os.path.join(budapest_results, '{A}_cnx_isc.npy'.format(A=A)))),axis=0)

isc_p, clf_p, cnx_p = [run_permutations(d, 10000) for d in [isc,clf, cnx]]
for d,typ in zip([isc_p,clf_p,cnx_p],['isc','clf','cnx']):
    for key in d.keys():
        results_dict['dataset'].append('budapest')
        results_dict['metric'].append(typ)
        results_dict['comparison'].append(key)
         # these are always one-sided because we expect HA to outperform AA
        if 'aa' in key:
            d[key]=2*d[key]
        results_dict['pvalue'].append(d[key])
    

### Raiders 

In [8]:
results = '../../final_results/raiders/'
isc={}
for A in ['aa','cha','rha','h2a']:
    isc[A] = np.mean(np.nan_to_num(np.load(os.path.join(results, '{A}_vertex_isc.npy'.format(A=A)))),axis=0)

clf={}
for A in ['aa','cha','rha','h2a']:
    clf[A] = np.mean(np.nan_to_num(np.load(os.path.join(results, '{A}_clf.npy'.format(A=A)))),axis=1)

cnx={}
for A in ['aa','cha','rha','h2a']:
    cnx[A] = np.mean(np.nan_to_num(np.load(os.path.join(results, '{A}_cnx_isc.npy'.format(A=A)))),axis=0)

In [9]:
isc_p, clf_p, cnx_p = [run_permutations(d, 10000) for d in [isc,clf, cnx]]
for d,typ in zip([isc_p,clf_p,cnx_p],['isc','clf','cnx']):
    for key in d.keys():
        results_dict['dataset'].append('raiders')
        results_dict['metric'].append(typ)
        results_dict['comparison'].append(key)
         # these are always one-sided because we expect HA to outperform AA
        if 'aa' in key:
            d[key]=2*d[key]
        results_dict['pvalue'].append(d[key])

### Whiplash

In [29]:
results = '../../final_results/whiplash/'
isc={}
for A in ['aa','cha','rha','h2a']:
    isc[A] = np.mean(np.nan_to_num(np.load(os.path.join(results, '{A}_vertex_isc.npy'.format(A=A)))),axis=0)

clf={}
for A in ['aa','cha','rha','h2a']:
    clf[A] = np.mean(np.nan_to_num(np.load(os.path.join(results, '{A}_clf.npy'.format(A=A)))),axis=1)
clf['aa'] = np.mean(np.nan_to_num(np.load(os.path.join(results, '{A}_clf.npy'.format(A='aa')))),axis=1)

cnx={}
for A in ['aa','cha','rha','h2a']:
    cnx[A] = np.mean(np.nan_to_num(np.load(os.path.join(results, '{A}_cnx_isc.npy'.format(A=A)))),axis=0)

isc_p, clf_p, cnx_p = [run_permutations(d, 10000) for d in [isc,clf,cnx]]
for d,typ in zip([isc_p,clf_p,cnx_p],['isc','clf','cnx']):
    for key in d.keys():
        results_dict['dataset'].append('whiplash')
        results_dict['metric'].append(typ)
        results_dict['comparison'].append(key)
        # these are always one-sided because we expect HA to outperform AA
        if 'aa' in key:
            d[key]=2*d[key]
        results_dict['pvalue'].append(d[key])    
    
res = pd.DataFrame(results_dict)
res.to_csv('../../final_results/significance_all_datasets_final.csv')    